In [1]:
import numpy as np
import pandas as pd
import PIL
import matplotlib.patches as patches
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Dropout,Flatten,Conv2D,MaxPooling2D
from tensorflow.keras.preprocessing import image

In [2]:
attr_data_path = "list_attr_celeba.csv"
images_data_path = "C:/Users/foreh/Documents/celeba/greyscale"
images_data_size = 10000 

tf.keras.backend.clear_session()
width = 178
height = 218
x_ = width/2                            # new value of x
image_size_ratio = width / height    # dimensions ratio
y_ = height/2

original_image_size = (width,height)
new_image_size = (int(width/2),int(height/2))
df_org = pd.read_csv(attr_data_path)

model = Sequential()
model.add(tf.keras.Input(shape=(new_image_size[1], new_image_size[0],1)))
model.add(tf.keras.layers.Normalization())
model.add(Dropout(0.1))
model.add(Conv2D(filters=64, kernel_size=(3,3), padding='same', activation="relu",kernel_regularizer='l2'))
model.add(tf.keras.layers.Normalization())
model.add(Dropout(0.1))
model.add(MaxPooling2D(pool_size=5))
model.add(Conv2D(filters=128, kernel_size=(5,5), padding='same', activation='relu',kernel_regularizer='l2'))
model.add(tf.keras.layers.Normalization())
model.add(Dropout(0.1))
model.add(MaxPooling2D(pool_size=2))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding='same', activation='relu',kernel_regularizer='l1'))
model.add(tf.keras.layers.Normalization())
model.add(Dropout(0.1))
model.add(MaxPooling2D(pool_size=2))
model.add(Flatten())
model.add(Dense(256, activation='relu',kernel_regularizer='l2'))
model.add(Dense(512, activation='relu',kernel_regularizer='l2'))
model.add(Dense(2, activation='relu',kernel_regularizer='l2'))
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 normalization (Normalizatio  (None, 109, 89, 1)       3         
 n)                                                              
                                                                 
 dropout (Dropout)           (None, 109, 89, 1)        0         
                                                                 
 conv2d (Conv2D)             (None, 109, 89, 64)       640       
                                                                 
 normalization_1 (Normalizat  (None, 109, 89, 64)      129       
 ion)                                                            
                                                                 
 dropout_1 (Dropout)         (None, 109, 89, 64)       0         
                                                                 
 max_pooling2d (MaxPooling2D  (None, 21, 17, 64)       0

In [3]:
from keras import backend as K

def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(tf.cast(y_true,tf.float32) * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (tf.cast(possible_positives,tf.float32) + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(tf.cast(y_true,tf.float32) * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (tf.cast(predicted_positives,tf.float32) + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

# Our custom loss' wrapper
def custom_loss():

    def recall_spec_loss(y_true, y_pred):
        return 1/f1_m(y_true, y_pred)

    # Returns the (y_true, y_pred) loss function
    return recall_spec_loss

In [4]:
for i in range(1, 21):
    if (i > 1):
        model = tf.keras.models.load_model('cnn17_skewed/iter'+str(i-1))
        print("model loaded: iter"+str(i-1))
        print("training model "+str(i))
        model.evaluate(X_val,y_val)
    df = df_org[(1+images_data_size*i):images_data_size*(i+1)]
    df = df.replace(-1,0)
    #print(df.head(5))
    images_data = list()
    for idx in range(df.shape[0]):
        path = "{}/{}".format(str(images_data_path),str(df.iloc[idx].image_id))
        image = PIL.Image.open(path).resize(new_image_size)
        image_array = np.asarray(image)
        images_data.append(image_array)
    images_data = np.array(images_data)
    def image_from_index(index, size=(original_image_size), path=images_data_path):
        path = "{}/{}".format(str(path),str(df.iloc[index].image_id))
        # to read the image
        image = PIL.Image.open(path).resize(size)
        image_array = np.asarray(image)

        return image_array
    images = []
    images = images_data # list of array of images
    labels = df[["image_id", "Attractive", "Young"]]  # dataframe of image features
    train_images,test_images,train_labels,test_labels = train_test_split(images,labels,test_size=0.3,random_state=(i%6))
    y_test=test_labels.drop(["image_id"],axis=1)
    y_train=train_labels.drop(["image_id"],axis=1)

    X_test=test_images
    X_train=train_images
    print("Samples Training Set:",len(X_train))
    X_test, X_val, y_test, y_val=train_test_split(X_test, y_test,test_size=0.5,random_state=(i%5))
    print("Samples in Validation Data:",len(X_val))
    print("Samples in Test Data:",len(X_test))
    x_ = new_image_size[0]
    y_ = new_image_size[1]
    BATCH_SIZE = (4)
    EPOCHS = 4
    print("Batch size: " + str(BATCH_SIZE))
    if False:
          print("SGD")
          opt = tf.keras.optimizers.SGD(learning_rate=0.001)
          model.compile(optimizer = opt,loss="binary_crossentropy",metrics = [tf.keras.metrics.Precision(), tf.keras.metrics.Recall(), 'binary_accuracy'])
    else:
        #precision_m, recall_m, f1_m, 
        print("ADAM")
        opt = tf.keras.optimizers.Adam(learning_rate=0.001)
        model.compile(optimizer = opt,loss="binary_crossentropy",metrics = ['binary_accuracy'])
    training_process = model.fit(X_train,y_train,epochs = EPOCHS,validation_data = (X_val,y_val),batch_size=BATCH_SIZE,shuffle = True)
    model.save('cnn17_skewed/iter'+str(i))
    print("Model " + str(i) +"evaluating:")
    model.evaluate(X_val,y_val)
    tf.keras.backend.clear_session()

Samples Training Set: 6999
Samples in Validation Data: 1500
Samples in Test Data: 1500
Batch size: 4
ADAM
Epoch 1/4
1750/1750 [==============================] - 30s 15ms/step - loss: 7.1990 - binary_accuracy: 0.6327 - val_loss: 6.3873 - val_binary_accuracy: 0.6293
Epoch 2/4
1750/1750 [==============================] - 27s 15ms/step - loss: 5.4343 - binary_accuracy: 0.6257 - val_loss: 4.9809 - val_binary_accuracy: 0.6293
Epoch 3/4
1750/1750 [==============================] - 27s 15ms/step - loss: 4.9064 - binary_accuracy: 0.6294 - val_loss: 4.9862 - val_binary_accuracy: 0.6293
Epoch 4/4
1750/1750 [==============================] - 27s 15ms/step - loss: 4.9043 - binary_accuracy: 0.6350 - val_loss: 4.9754 - val_binary_accuracy: 0.6293
INFO:tensorflow:Assets written to: cnn17_skewed/iter1\assets
Model 1evaluating:
47/47 [==============================] - 1s 19ms/step - loss: 4.9754 - binary_accuracy: 0.6293
model loaded: iter1
training model 2
47/47 [==============================] - 1s 17

KeyboardInterrupt: 

In [38]:
attr_data_path = "list_attr_celeba.csv"
images_data_path = "C:/Users/foreh/Documents/celeba/greyscale"

tf.keras.backend.clear_session()
width = 178
height = 218
df_org = pd.read_csv(attr_data_path)

df = df_org[190000:202600]
df = df.replace(-1,0)
#print(df.head(5))
images_data = list()
for idx in range(df.shape[0]):
    path = "{}/{}".format(str(images_data_path),str(df.iloc[idx].image_id))
    image = PIL.Image.open(path)
    image_array = np.asarray(image)
    images_data.append(image_array)
images_data = np.array(images_data)
def image_from_index(index, size=(width,height), path=images_data_path):
    path = "{}/{}".format(str(path),str(df.iloc[index].image_id))
    # to read the image
    image = PIL.Image.open(path).resize(size)
    image_array = np.asarray(image)

    return image_array
images = images_data # list of array of images
labels = df[["image_id", "Bangs", "Straight_Hair", "Wavy_Hair", "Wearing_Hat"]]  # dataframe of image features
labels=labels.drop(["image_id"],axis=1)

In [10]:
for i in range(1, 19):
    tf.keras.backend.clear_session()
    model1 = tf.keras.models.load_model('cnn8_checkpoints/iter'+str(i))
    model1.evaluate(images,labels)

394/394 [==============================] - 13s 32ms/step - loss: 0.3009 - precision: 0.7161 - recall: 0.5216 - binary_accuracy: 0.8682


In [14]:
## MODEL DESIGN AREA
tf.keras.backend.clear_session()



model = Sequential()
model.add(tf.keras.Input(shape=(new_image_size[1], new_image_size[0],1)))
model.add(tf.keras.layers.Normalization())
model.add(Dropout(0.1))
model.add(Conv2D(filters=64, kernel_size=(3,3), padding='same', activation="relu",kernel_regularizer='l2'))
model.add(tf.keras.layers.Normalization())
model.add(Dropout(0.1))
model.add(MaxPooling2D(pool_size=2))
model.add(Conv2D(filters=128, kernel_size=(5,5), padding='same', activation='relu',kernel_regularizer='l2'))
model.add(tf.keras.layers.Normalization())
model.add(Dropout(0.1))
model.add(MaxPooling2D(pool_size=2))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding='same', activation='relu',kernel_regularizer='l2'))
model.add(tf.keras.layers.Normalization())
model.add(Dropout(0.1))
model.add(MaxPooling2D(pool_size=2))
model.add(Flatten())
model.add(Dense(256, activation='relu',kernel_regularizer='l2'))
model.add(Dense(512, activation='relu',kernel_regularizer='l2'))
model.add(Dense(4, activation='relu',kernel_regularizer='l2'))
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 normalization (Normalizatio  (None, 109, 89, 1)       3         
 n)                                                              
                                                                 
 dropout (Dropout)           (None, 109, 89, 1)        0         
                                                                 
 conv2d (Conv2D)             (None, 109, 89, 64)       640       
                                                                 
 normalization_1 (Normalizat  (None, 109, 89, 64)      129       
 ion)                                                            
                                                                 
 dropout_1 (Dropout)         (None, 109, 89, 64)       0         
                                                                 
 max_pooling2d (MaxPooling2D  (None, 54, 44, 64)       0